# 0. Install Dependencies

In [ ]:
%pip install gym==0.25.2 tensorflow keras-rl2 pygame==2.6.0 numpy==1.26.0

  Using cached tensorflow-2.15.1-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-win_amd64.whl.metadata (20 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached tensorflow-2.15.1-cp311-cp311-win_amd64.whl (2.1 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
Using cached ml_dtypes-0.3.2-cp311-cp311-win_amd64.whl (127 kB)
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB 10.2 MB/s eta 0:00:02
   -- ------------------------------------- 1.2/15.8 MB 14.8 MB/s eta 0:00:01
   --- ------------------------------------ 1.3/15.8 MB 11.5 MB/s eta 0:00:02


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. Test Random Environment with OpenAI Gym

In [7]:
import gym 
import random

In [8]:
env = gym.make('CartPole-v0', render_mode="human")
states = env.observation_space.shape[0]     # number of states available in environment
actions = env.action_space.n    # actions available

In [9]:
actions

2

In [10]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    print(f"Inside Episode: {episode}")
    
    while not done:
        env.render()        # render the env so we can see it 
        action = random.choice([0,1])       # taking random step of left or right
        n_state, reward, terminated, truncated, info = env.step(action)      # actually taking the step and getting new states, reward, status of the game (whether it ended naturally "terminated" or forcibly stopped "truncated"), other info
        score+=reward       # accumulating reward
    print('Episode:{} Score:{}'.format(episode, score))

DependencyNotInstalled: pygame is not installed, run `pip install gym[classic_control]`

# 2. Create a Deep Learning Model with Keras

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(states, actions):   # just defining a standard sequential model
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))      # define paths based on each action available
    return model

In [20]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent      # Just one of the many different Agents you can use to train model (NAF, DDPG, SARSA are some others)
from rl.policy import BoltzmannQPolicy      # Type of strategy to choose action: select from probability dist rather than always the highest Q-value
from rl.memory import SequentialMemory      # Just memeory for agent to remember what it did b4

### Main VS. Target Q-Network
Q-values are action-value functions and that represents the "expected cumulative reward" for a particular action the agent takes in a given state, provided that they only follow an optimal policy after that. 
When training a Q-network we have to update the q-values but if we update it, the second part of "following an optimal policy" can change which can change the "expected cumulative reward".

Solution: Instead of using the current Q-network for both evaluation and target estimation, we use a separate network (target Q-network) that updates more slowly.
    - Main Q-Network: Used for predicting Q-values and selecting actions.
    - Target Q-Network: Used for computing stable target Q-values.

In [ ]:
def build_agent(model, actions):        # function defining agent
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)     # target_model_update defines "target Q-network will update every 0.01 steps." see above about target Q-network.
    return dqn

In [ ]:
# instantiate, compile and fit
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
51 episodes - episode_reward: 193.118 [58.000, 200.000] - loss: 5.376 - mae: 39.195 - mean_q: 78.705

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: 1.0000
54 episodes - episode_reward: 185.056 [64.000, 200.000] - loss: 7.553 - mae: 40.527 - mean_q: 81.044

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: 1.0000
52 episodes - episode_reward: 193.462 [33.000, 200.000] - loss: 8.935 - mae: 40.588 - mean_q: 81.322

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 11.357 - mae: 41.679 - mean_q: 83.338

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.0000
done,

In [27]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)       # this just runs the testing again but with the visualization

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)     # Save weights into this file

In [ ]:
# Cleanup
del model
del dqn
del env

In [ ]:
# Reinitialization and compilation of dqn
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
# Load the weights back in
dqn.load_weights('dqn_weights.h5f')

In [11]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
